# __STRATEGIE D'ANALYSE BIOINFORMATIQUE - MENTORING PROJECT__

__AGMIMONHAN Attolou Raoul, NAME Pakyendou Estel__

__Tuteurs: Aurore COMTE & Sebastien RAVEL__

Jupyter inspired by the model created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD)
***

# <span style="color: #006E7F">Table of contents</span>
<a class="anchor" id="home"></a>


[PRACTICE IV - SNP calling](#SNP)
   * [Indexation reference genome `samtools faidx`](#refindex)
   * [Generer bcf file `bcftools mpileup`](#bcftools)
   * [Calling `bcftools call`](#calling)
   * [SNP statistiques `bcftools stats`](#stats)
   * [SNP annotation](#annotation)
   
   

***

# __Practice IV - SNP calling__

## __IV.1. Indexation reference genome__

In [ ]:
## Se deplacer dans le répertoire de la référennce

cd /scratch/MOryzae/REF

In [ ]:
# Charger le module samtools 

module load samtools/1.18


In [ ]:
## Indexation de la référence utilisant samtools faidx

samtools faidx /scratch/MOryzae/REF/MOryzae_genomic.fna

In [ ]:
## Vérifier le contenu du répertoire et les fichiers générés

ls -lh

## __IV.2. RUN__

In [ ]:
## Se déplacer dans le répertoire créé

cd /scratch/MOryzae/SCRIPTS

In [ ]:
## Ouvrir l'éditeur de texte nano

nano snp_calling_pipeline.sh

In [ ]:
#!/bin/bash

############ SLURM Configuration ##############

### Define Job name
#SBATCH --job-name=snp2_calling

### Define partition to use
#SBATCH -p normal

### Define number of CPUs to use
#SBATCH -c 16

### Specify the node to run on
#SBATCH --nodelist=node20  # Run the job on node20

#################################################

########### Execution Commands ###################

# Variables
SORTED_PATH="/scratch/MOryzae/MAPPING/bam_mapped_sort"
VCF_OUTPUT="/scratch/MOryzae/SNP/vcf_files/all_samples.vcf.gz"
SNP_OUTPUT="/scratch/MOryzae/SNP/vcf_files/snp.vcf.gz"
REF_GENOME="/scratch/MOryzae/REF/MOryzae_genomic.fna"
SNP_STATS_DIR="/scratch/MOryzae/SNP/stats"

# Load necessary modules
module load samtools/1.18
module load bcftools/1.18

# Create necessary directories
mkdir -p /scratch/MOryzae/SNP/vcf_files "$SNP_STATS_DIR"

# Step 1: Generate VCF file, compress, and index
echo -e "######################\nGenerating compressed VCF file"
bcftools mpileup -Ou --threads 16 -f "$REF_GENOME" "$SORTED_PATH"/*.mappedpaired.sorted.bam | \
  bcftools call -mv -Oz -o "$VCF_OUTPUT" || {
    echo "Error: VCF generation and compression failed" >&2
    exit 1
}

# Step 2: Index the compressed VCF file
echo -e "######################\nIndexing compressed VCF file"
bcftools index "$VCF_OUTPUT" || {
    echo "Error: VCF file indexing failed" >&2
    exit 1
}

# Step 3: Filter to retain only SNPs
echo -e "######################\nFiltering to retain only SNPs"
bcftools view -v snps -Oz -o "$SNP_OUTPUT" "$VCF_OUTPUT" || {
    echo "Error: SNP filtering failed" >&2
    exit 1
}

# Step 4: Index the SNP VCF file
echo -e "######################\nIndexing SNP VCF file"
bcftools index "$SNP_OUTPUT" || {
    echo "Error: SNP VCF file indexing failed" >&2
    exit 1
}

# Step 5: Generate SNP statistics
echo -e "######################\nGenerating SNP statistics"
bcftools stats "$SNP_OUTPUT" > "$SNP_STATS_DIR/all_samples_SNP_statistics.txt" || {
    echo "Error: Failed to generate SNP statistics" >&2
    exit 1
}

echo "Compressed VCF file and SNP-specific VCF file generated and indexed successfully."


In [ ]:
## Lancer le script

sbash snp_calling_pipeline.sh

### Correction de l'identité de chaque isolat dans les fichiers vcf

In [ ]:
## Se déplacer dans le répertoire SNP

cd /scratch/MOryzae/SNP/vcf_files

In [ ]:
## Charger le module bcftools

module load bcftools/1.18

In [ ]:
## Utilisez bcftools pour lister les noms des échantillons actuels :

bcftools query -l snp.vcf.gz> snp_samples.txt

Cela créera un fichier samples.txt contenant les noms des échantillons actuels (tels que ceux avec des chemins et suffixes).

In [ ]:
## Modifiez le fichier samples.txt ou créez un nouveau fichier, par exemple new_samples.txt, 
## en mappant les anciens noms aux nouveaux noms simplifiés.

awk -F'/' '{print $0 "\t" $NF}' snp_samples.txt | sed 's/.mappedpaired.sorted.bam//g' > new_snp_samples.txt

In [ ]:
## Récuper rien que les noms des isolats dans un nouveau fichier texte


awk -F'/' '{print $0 "\t" $NF}' new_snp_samples.txt | cut -f3 > new2_snp_samples.txt

In [ ]:
### Utilisez bcftools reheader pour appliquer les modifications :

bcftools reheader -s new2_snp_samples.txt -o snp_correct.vcf.gz snp.vcf.gz

Vérifier les modifications

In [ ]:
## Reindexer les nouveaux fichiers vcf.gz

bcftools index snp_correct.vcf.gz

### Récupérer tous les outputs générés dans le répertoire SNP sur le NAS

In [ ]:
## Déplacer tout le répertoire MAPPING vers le NAS

scp -r /scratch/MOryzae/SNP san:/projects/medium/CIBiG_MOryzae/